# Clasificador de spam

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Recopilado y limpieza de texto
Se toman dos archivos de texto plano:


*   Uno que contiene las palabras caracteristicas de spam
*   Uno que contiene las palabras caracteristicas de ham

Ambos son procesados y limpiados para generar una lista de palabras.



In [0]:
def limpiatexto(texto):
	for c in texto:
		if c in ['.', '!', '?', '\n', '\t', '-', '¿', '¡', ':']:
			texto = texto.replace(c, ' ')
		if ord(c) in range(ord('a'), ord('z') + 1) or ord(c) in range(ord('A'), ord('Z') + 1) or c in [' ', 'á', 'é', 'í', 'ó', 'ú', 'Á', 'É', 'Í', 'Ó', 'Ú']:
			pass
		else:
			texto = texto.replace(c, '')
	texto = texto.lower()
	return texto

def limpialista(lista):
	aux = list()
	for l in lista:
		l.replace(' ','')
		if l != '':
			aux.append(l)
	return aux

def generapalabras(doc):
    texto = ''
    for l in doc.readlines():
        texto += l
    texto = limpiatexto(texto)
    p = texto.split(' ')
    p = limpialista(p)
    return p

In [0]:
spam = open('/content/drive/My Drive/Colab Notebooks/Clasificador de spam/spamesp.txt', 'r')
ham = open('/content/drive/My Drive/Colab Notebooks/Clasificador de spam/hamesp.txt', 'r')
p_spam = generapalabras(spam)
p_ham = generapalabras(ham)
spam.close()
ham.close()

In [4]:
p_spam[0:10]

['contácteme',
 'ahora',
 'para',
 'ganar',
 'hoy',
 'actúa',
 'ahora',
 'para',
 'mantener',
 'tu']

In [5]:
p_ham[0:10]

['seleccione',
 'los',
 'servicios',
 'que',
 'desea',
 'agregar',
 'también',
 'quiero',
 'hablar',
 'sobre']

## Creación del dataframe
Ahora, toca crear un dataframe de la librería **pandas** incluida en Python 3. Primero creamos dos diccionarios, uno para spam y otro para ham y a partir de estos, contruimos de una forma sencilla el dataframe.

In [0]:
import pandas as pd

def crearbanco(palabras):
	banco = {}
	for p in palabras:
		if p not in banco.keys():
			banco[p] = 1
		else:
			banco[p] += 1
	return banco

def creartabla(spam, ham):
	llaves = list(spam.keys()) + list(ham.keys())
	llaves = list(set(llaves))
	llaves.sort()
	counts, counth = list(), list()
	for k in llaves:
		if k in spam.keys():
			counts.append(spam[k])
		else:
			counts.append(0)
		if k in ham.keys():
			counth.append(ham[k])
		else:
			counth.append(0)
	dic = {'spam':counts, 'ham':counth}
	tabla = pd.DataFrame(dic)
	tabla.index = llaves
	return tabla

In [0]:
bancospam = crearbanco(p_spam)
bancoham = crearbanco(p_ham)
df_clasificador = creartabla(bancospam, bancoham)

In [8]:
df_clasificador.loc[['por', 'favor', 'dame', 'amor']]

,spam,ham
por,37,105
favor,8,54
dame,0,3
amor,1,0


Podemos exportar el dataframe a un archivo de extensión .csv si es necesario.

## Clasificación a través de Naive Bayes y aplanado Laplaciano
Obtenemos la probabilidad de que nuestro mensaje sea spam o ham utilizando el teorema de Bayes. Es importante recalcar que el uso de logaritmos es necesario ya que así se elimina el problema del **underflow**.

![texto alternativo](https://latex.codecogs.com/gif.latex?P%5Cleft%20%28%20spam%7CM%20%5Cright%20%29%3D%20%5Cfrac%7BP%5Cleft%20%28M%7Cspam%20%5Cright%20%29P%28spam%29%7D%7BP%28M%29%7D)

![texto alternativo](https://latex.codecogs.com/gif.latex?P%5Cleft%20%28%20ham%7CM%20%5Cright%20%29%3D%20%5Cfrac%7BP%5Cleft%20%28M%7Cham%20%5Cright%20%29P%28ham%29%7D%7BP%28M%29%7D)

El problema del sobreajuste se soluciona implementando el **aplanado Laplaciano**.

![texto alternativo](https://latex.codecogs.com/gif.latex?P%5Cleft%20%28x%20%3D%20e%20%5Cright%20%29%3D%20%5Cfrac%7Bcount%28x%3De%29%20&plus;%20k%7D%7B%7Ce%7C%20&plus;%20kx%7D)

In [0]:
import math

def clasifica(pal, df, sumas, k):
    cad = ' '.join(pal)
    probspam = math.log(df['spam'].gt(0).sum()+k)-math.log(len(list(df.index))+k*2)
    probham = math.log(df['ham'].gt(0).sum()+k)-math.log(len(list(df.index))+k*2)
    for p in pal:
        if p in list(df.index):
            probspam += math.log(list(df.loc[p])[0]+k)-math.log(sumas[0]+k*len(list(df.index)))
            probham += math.log(list(df.loc[p])[1]+k)-math.log(sumas[1]+k*len(list(df.index)))
    res = max([probspam, probham])
    if res == probspam:
        return 'Tu mensaje es spam'
    else:
        return 'Tu mensaje es ham'

def resultado(doc, df, k):
    cad = ''
    p_input = generapalabras(doc)
    doc.seek(0)
    for l in doc.readlines():
        cad += l
    salida = clasifica(p_input, df, list(df.sum()), k)
    return cad + '\n' + salida

In [12]:
entrada = open('/content/drive/My Drive/Colab Notebooks/Clasificador de spam/entrada1.txt', 'r')
print(resultado(entrada, df_clasificador, 1))
entrada.close()

Boris dame un abrazo muy fuerte.
Tu mensaje es ham


In [11]:
entrada2 = open('/content/drive/My Drive/Colab Notebooks/Clasificador de spam/entrada2.txt', 'r')
print(resultado(entrada2, df_clasificador, 1))
entrada2.close()

HAGA CLICK EN ESTE ENLACE PARA GANAR UN PREMIO.
Tu mensaje es spam
